In [1]:
import mxnet as mx
import mxnet.ndarray as nd
from mxnet import gluon
from mxnet import autograd

In [2]:
class ReluOp(mx.operator.CustomOp):
    
    guided_backprop = True
    
    def forward(self, is_train, req, in_data, out_data, aux):
        x = in_data[0]
        y = nd.maximum(x, nd.zeros_like(x))
        self.assign(out_data[0], req[0], y)

    def backward(self, req, out_grad, in_data, out_data, in_grad, aux):
        if ReluOp.guided_backprop:
            y = out_data[0]
            dy = out_grad[0]
            dy_positives = nd.maximum(dy, nd.zeros_like(dy))
            y_ones = y.__gt__(0)
            dx = dy_positives * y_ones
            self.assign(in_grad[0], req[0], dx)
        else:
            x = in_data[0]
            x_gt_zero = x.__gt__(0)
            dx = out_grad[0] * x_gt_zero
            self.assign(in_grad[0], req[0], dx)

@mx.operator.register("relu")
class ReluProp(mx.operator.CustomOpProp):
    def __init__(self):
        super(ReluProp, self).__init__(True)

    def infer_shape(self, in_shapes):
        data_shape = in_shapes[0]
        output_shape = data_shape
        return (data_shape,), (output_shape,), ()

    def create_operator(self, ctx, in_shapes, in_dtypes):
        return ReluOp()  

class Activation(mx.gluon.HybridBlock):
    @staticmethod
    def set_guided_backprop(mode=True):
        ReluOp.guided_backprop = mode
    
    def __init__(self, act_type, **kwargs):
        assert act_type == 'relu'
        super(Activation, self).__init__(**kwargs)

    def hybrid_forward(self, F, x):
        return F.Custom(x, op_type='relu')


In [5]:
r = Activation('relu')
x = mx.nd.array([1, 2, -4, -5])

Activation.set_guided_backprop(False)

x.attach_grad()
with autograd.record():
    y = r(x)
    print(y)

y.backward(nd.array([-1,2,3,-4]))
#y.backward(nd.array([1,1,1,1]))

x.grad


[1. 2. 0. 0.]
<NDArray 4 @cpu(0)>



[-1.  2.  0. -0.]
<NDArray 4 @cpu(0)>

In [ ]:
mx.nd.array([-7]) * mx.nd.array([-0])